In [1]:
%load_ext lab_black

import pandas as pd

In [1]:
import numpy as np
np.log(2)

0.6931471805599453

In [2]:
typecodes = (
    pd.read_parquet("/data/data/open_dataset/oag/2018.parquet")
    .groupby(["GeneralAcft", "GeneralAcftName"])[["Km"]]
    .count()
    .reset_index()
    .rename(
        columns={
            "GeneralAcft": "typecode",
            "GeneralAcftName": "typename",
            "Km": "flights",
        }
    )
    .sort_values("flights", ascending=False)
)
typecodes.to_csv("typecodes_raw.csv", index=False)
print(sum(typecodes.flights), "flights")
typecodes

47282699 flights


,typecode,typename,flights
7,32S,Airbus A318/319/320/321,11991138
22,737,Boeing 737 Passenger,10459732
112,RFS,Road Feeder Service (Truck),5591470
87,EMJ,Embraer 170/195,2481856
122,TRN,Train,2452475
...,...,...,...
117,SH6,Shorts 360 (sd3-60),40
96,I14,Ilyushin IL114,32
128,YN7,Xian Yunshuji Y7/MA60,32
35,A38,Antonov AN-38,26


In [2]:
typecodes = pd.read_csv("typecodes_raw.csv")
typecodes.head(20)

,typecode,typename,flights
0,32S,Airbus A318/319/320/321,11991138
1,737,Boeing 737 Passenger,10459732
2,RFS,Road Feeder Service (Truck),5591470
3,EMJ,Embraer 170/195,2481856
4,TRN,Train,2452475
5,CRJ,Canadair Regional Jet,2249699
6,ATR,ATR42/ATR72,1651502
7,DH8,DHvilld-Bombardier DHC8 Dsh 8,1259419
8,777,Boeing 777 Passenger,868711
9,330,Airbus A330,821600


In [4]:
# typecode duplicates
typecodes.groupby("typecode")[["typename"]].count().reset_index().query(
    "typename>1"
).merge(typecodes, on="typecode")

,typecode,typename_x,typename_y,flights
0,310,2,Airbus Industrie A310 Passenger,16465
1,310,2,Airbus A310 Passenger,3817
2,31F,2,Airbus A310 Freighter,6702
3,31F,2,Airbus Industrie A310 Freighter,104
4,32S,2,Airbus A318/319/320/321,11991138
5,32S,2,Airbus Industrie A318/319/320/321,125061
6,330,2,Airbus A330,821600
7,330,2,Airbus Industrie A330,197591
8,33F,2,Airbus A330 Freighter,18794
9,33F,2,Airbus Industrie A330 Freighter,11167


In [5]:
# Duplicate IATA ac codes
# "CSB": "BCS1/BCS3", # IATA CS1/CS3
display(typecodes.query("typecode=='CSB'"))
display(typecodes.query("typecode=='CS3'"))

,typecode,typename,flights
37,CSB,Bombardier C Series,66414


,typecode,typename,flights


In [3]:
typecodes = (
    typecodes.groupby("typecode")
    .sum()
    .reset_index()
    .merge(typecodes, on="typecode")
    .drop("flights_y", axis=1)
    .rename(columns={"flights_x": "flights"})
    .drop_duplicates(["typecode"])
).reset_index(drop=True)
typecodes

,typecode,flights,typename
0,100,98059,Fokker 100
1,146,30864,British Aerospace 146 Passenger
2,14F,1753,British Aerospace 146 (Freighter)
3,310,20282,Airbus Industrie A310 Passenger
4,31F,6806,Airbus A310 Freighter
...,...,...,...
113,TU5,290,Tupolev TU154
114,YK2,5410,Yakovlev Yak-42/142
115,YK4,744,Yakovlev Yak-40
116,YN2,3471,Harbin Yunshuji Y12


In [4]:
aircraft_type_designators = pd.read_csv("aircraft_type_designators.csv", index_col=[0])
aircraft_type_designators

,ac_code_iata,ac_code_icao,bada_model,ac_name_oag,ac_name_ps
0,A40,A140,A140,Antonov AN-140,Antonov An-140
1,A81,A148,A148,Antonov An148-100,Antonov An-148
2,A58,A158,A158,Antonov An-158,Antonov An-158
3,32N,A20N,A20N,Airbus A320neo,Airbus A320neo
4,32Q,A21N,A21N,Airbus A321neo,Airbus A321neo
...,...,...,...,...,...
128,TU5,T154,T154,Tupolev Tu154,Tupolev Tu-154
129,T20,T204,T204,Tupolev Tu-204/Tu-214,Tupolev Tu-204
130,YN2,Y12,C208,Harbin Yunshuji Y12,NaN
131,YK4,YK40,YK40,Yakovlev Yak-40,NaN


In [5]:
merge = (
    typecodes.merge(
        aircraft_type_designators,
        how="left",
        left_on="typecode",
        right_on="ac_code_iata",
    ).rename(
        columns={
            "ac_code_icao": "icao",
        }
    )[
        ["typecode", "icao", "typename", "flights"]
    ]
).dropna()
print(len(merge))
merge.head()

42


,typecode,icao,typename,flights
0,100,F100,Fokker 100,98059
11,717,B712,Boeing 717-200,283269
27,A38,AN38,Antonov AN-38,26
28,A40,A140,Antonov AN-140,2070
30,A58,A158,Antonov An-158,1276


In [6]:
mapping = {
    # Airbus
    "ABF": "A306",
    "AB3": "A306/A30B",
    "31F": "A310",
    "32S": "A318/A319/A320/A321",
    "330": "A332/A333/A339",
    "340": "A342/A343/A345/A346",
    "350": "A359/A35K",
    "380": "A388",
    "33F": "A332",
    "310": "A310",
}
mapping = mapping | {
    # Boeing
    "727": "B722",
    "72F": "B722",
    "737": "B38M/B39M/B732/B733/B734/B735/B736/B737/B738/B739",
    "73M": "B732/B733/B734/B737",  # Mixed conf
    "73F": "B732/B733/B734/B738",  # Freighter
    "747": "B744/B748",
    "74M": "B744",
    "74F": "B744",
    "757": "B752/B753",
    "75F": "B752",
    "767": "B762/B763/B764",
    "76F": "B763",
    "777": "B772/B773/B77L/B77W",
    "77F": "B77L",
    "787": "B788/B789/B78X",
}
mapping = mapping | {
    # ATR
    "ATR": "AT43/AT72",
    "ATZ": "AT43",
    "ATF": "AT72",
}
mapping = mapping | {
    # to be removed (TBR) (helicopoters, train, bus, truck, limousines...
    "BH2": "TBR",  # helico
    "NDE": "TBR",  # helico
    "AWH": "TBR",  # helico
    "AGH": "TBR",  # helico
    "EC3": "TBR",  # helico
    "S76": "TBR",  # helico
    "LCH": "TBR",  # launch/boat
    "RFS": "TBR",  # Road Feeder Service (Truck)
    "TRN": "TBR",  # train
    "TGV": "TBR",  # train
    "BUS": "TBR",  # bus
    "LMO": "TBR",  # limousine
    "LOH": "TBR",  # Lockheed L182/L282/L382 (L100) Hercules (not found)
    "A4F": "TBR",  # Antonov AN-124 (not found)
    "ANF": "TBR",  # Antonov AN-12 (not found)
    "SH3": "TBR",  # Shorts 330 (sd3-30) (not found)
    "CS2": "TBR",  # Casa C212/Nusantara NC-212 Aviocar (not found)
    "D6F": "TBR",  # Boeing (Douglas) DC6A/B/C Freighter (not found)
    "SH6": "TBR",  # Shorts 360 (sd3-60) (not found)
}
mapping = mapping | {
    "EMJ": "E170/E190",
    "CRJ": "CRJ1/CRJ2/CRJ7/CRJ9",
    "DH": "DH8A/DH8B/DH8C/DH8D",
    "ERJ": "E135/E145",
    "M80": "MD81/MD82/MD83/MD87/MD88/MD90",
    "DH8": "DH8A/DH8B/DH8C/DH8D",
    "JST": "JS31/JS32/JS41",
    "SU1": "SU95",
    "CSB": "BCS1/BCS3",
    "ARJ": "RJ1H/RJ85",
    "146": "B461/B462/B463",
    "DHB": "DHC2",
    "EPH": "E55P",
    "M1F": "B773",  # md11 (b773 openap synonym md11)
    "DHC": "DHC2/DHC3/DHC6/DHC7",  # ???
    "CVF": "CVLT",
    "HBA": "BE40",
    "C21": "AJ27",
    "D9F": "DC94",
    "DHO": "DHC3",
    "14F": "B461/B462/B463",
    "BNT": "BN2P",
    "CRF": "CRJ2",
    "AN6": "AN24/AN38",  # ???
    "CVR": "CVLT",
    "IL7": "IL96",  # ???
    "BE9": "BE20",
    "M11": "B773",  # md11 (b773 openap synonym md11)
    "I14": "SB20",
    "YN7": "F27",
}

In [39]:
typecodes.query("typecode=='YN7'")

,typecode,typename,flights
126,YN7,Xian Yunshuji Y7/MA60,32


In [ ]:
typecodes[~typecodes.typecode.isin(aircraft_type_designators.ac_code_iata)].iloc[75:80]

In [ ]:
aircraft_type_designators.query("ac_name_oag.str.contains('Nusan')")

In [7]:
mapping = mapping | dict(
    (typecode, icao) for _, (typecode, icao, _, _) in merge.iterrows()
)
mapping

{'ABF': 'A306',
 'AB3': 'A306/A30B',
 '31F': 'A310',
 '32S': 'A318/A319/A320/A321',
 '330': 'A332/A333/A339',
 '340': 'A342/A343/A345/A346',
 '350': 'A359/A35K',
 '380': 'A388',
 '33F': 'A332',
 '310': 'A310',
 '727': 'B722',
 '72F': 'B722',
 '737': 'B38M/B39M/B732/B733/B734/B735/B736/B737/B738/B739',
 '73M': 'B732/B733/B734/B737',
 '73F': 'B732/B733/B734/B737',
 '747': 'B744/B748',
 '74M': 'B744',
 '74F': 'B744',
 '757': 'B752/B753',
 '75F': 'B752',
 '767': 'B762/B763/B764',
 '76F': 'B763',
 '777': 'B772/B773/B77L/B77W',
 '77F': 'B77L',
 '787': 'B788/B789/B78X',
 'ATR': 'AT43/AT72',
 'ATZ': 'AT43',
 'ATF': 'AT72',
 'BH2': 'TBR',
 'NDE': 'TBR',
 'AWH': 'TBR',
 'AGH': 'TBR',
 'EC3': 'TBR',
 'S76': 'TBR',
 'LCH': 'TBR',
 'RFS': 'TBR',
 'TRN': 'TBR',
 'TGV': 'TBR',
 'BUS': 'TBR',
 'LMO': 'TBR',
 'LOH': 'TBR',
 'A4F': 'TBR',
 'ANF': 'TBR',
 'SH3': 'TBR',
 'CS2': 'TBR',
 'D6F': 'TBR',
 'SH6': 'TBR',
 'EMJ': 'E170/E190',
 'CRJ': 'CRJ1/CRJ2/CRJ7/CRJ9',
 'DH': 'DH8A/DH8B/DH8C/DH8D',
 'ERJ': 'E

In [8]:
cumul = []
for _, (typecode, _, _) in typecodes.iterrows():
    if typecode in mapping.keys():
        for icao in mapping[typecode].split("/"):
            cumul.append((typecode, icao))
typecodes = pd.DataFrame.from_records(
    cumul, columns=["typecode", "icao"]
).drop_duplicates()
typecodes.to_csv("typecodes.csv", index=False)
typecodes

,typecode,icao
0,100,F100
1,146,B461
2,146,B462
3,146,B463
4,14F,B461
...,...,...
173,TU5,T154
174,YK2,YK42
175,YK4,YK40
176,YN2,Y12


In [48]:
counts = (
    pd.read_csv("typecodes.csv")
    .groupby("icao")
    .count()
    .reset_index()
    .sort_values("typecode", ascending=False)
)
counts[counts.typecode > 1]

,icao,typecode
118,TBR,19
36,B734,3
39,B737,3
34,B732,3
35,B733,3
50,B773,3
42,B744,3
51,B77L,2
89,F27,2
106,MD90,2


In [34]:
aircraft_type_designators[~aircraft_type_designators.ac_code_icao.isin(typecodes.icao)]

,ac_code_iata,ac_code_icao,bada_model,ac_name_oag,ac_name_ps
3,32N,A20N,A20N,Airbus A320neo,Airbus A320neo
4,32Q,A21N,A21N,Airbus A321neo,Airbus A321neo
62,BE2,BE55,PA27,Beechcraft-Lght Acft-Twin Piston Engine,NaN
67,CN2,C310,C310,Cessna Light Acft (Twin piston engines),NaN
68,CJ8,C680,C680,Cessna 680 Citation,NaN
73,CRK,CRJX,CRJX,Canadair (Bombardier) Regional Jet 1000,Bombardier CRJ-1000
93,E95,E195,E195,Embraer 195,Embraer ERJ-195
95,E7W,E75L,E75L,Embraer 175 (Enhanced Winglets),Embraer ERJ-175 EnhancedWinglets
96,E75,E75S,E75S,Embraer 175,Embraer ERJ-175 Base
100,CNT,F406,F406,Cessna Light Aircraft (Twin Turboprop,NaN
